#  Demo of Ridge Regression using Stochasitc Gradient Descent optimization

You will need to play with the `learning_rate` and `l2_lambda` (regularization parameter) hyperparameters in order to get a fast and stable optimizaiton.

* Small batches allow us to get quick estimates of the true gradient (since we only have to process one batch worth of data, instead of the whole data set).

* Randomization allows us to avoid pathalogical orderings of the data that might cause us to go off in the wrong direction.

* If we are solving a non-convex problem (e.g. not this case), then randomization can also help to avoid and escape local minima

In [2]:
from IPython.display import Latex


Our minimization problem is
$$
\min_\theta \mathcal{L}
$$
where our loss function is
$$
\mathcal{L} = \frac{1}{m} \|X\theta - y\|^2 + \lambda\|\theta\|^2
$$

The gradient is 
$$
\nabla_{\theta} \mathcal{L} = \frac{2}{m} X^T (X\theta - y) + 2\lambda\theta
$$

If we designate our error term as
$$
\hat{\varepsilon} = X\theta - y
$$
then we have
$$
\nabla_{\theta} \mathcal{L} = \frac{2}{m} X^T \hat{\varepsilon} + 2\lambda\theta
$$

This formulation corresponds directly to the implementations below



## On CPU using Pandas

Pandas + Numpy is often easiest for smaller problems

In [1]:
import numpy as np
import pandas as pd
import timeit

# Generate synthetic dataset
n_samples = 800
n_features = 50_000

# Features
X = pd.DataFrame(np.random.randn(n_samples, n_features))
X = (X - X.mean()) / X.std()

# True weights
true_weights = np.random.randn(n_features)

# Target variable with noise
y = pd.Series(X.values @ true_weights + 0.1 * np.random.randn(n_samples))


In [2]:


# Initialize weights
weights = np.zeros(n_features)
learning_rate = 1e-4
n_epochs = 50
batch_size = 32

# Timing start
start_time = timeit.default_timer()

# Training loop
for epoch in range(n_epochs):
  # Shuffle the data
  indices = np.random.permutation(n_samples)
  X_shuffled = X.values[indices]
  y_shuffled = y.values[indices]

  for i in range(0, n_samples, batch_size):
    X_batch = X_shuffled[i:i+batch_size]
    y_batch = y_shuffled[i:i+batch_size]

    # Predict
    y_pred = X_batch @ weights

    # Compute error
    error = y_pred - y_batch

    l2_lambda = 1e-3

    # Compute gradient
    gradient = X_batch.T @ error / batch_size + l2_lambda * weights

    # Update weights
    weights -= learning_rate * gradient

  # Compute loss
  y_full_pred = X.values @ weights
  loss = np.mean((y_full_pred - y.values) ** 2)
  print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

# Timing end
end_time = timeit.default_timer()

print(f"Training completed in {end_time - start_time:.2f} seconds")


Epoch 1, Loss: 36228.0376
Epoch 2, Loss: 25689.1424
Epoch 3, Loss: 18248.4911
Epoch 4, Loss: 12984.1272
Epoch 5, Loss: 9254.2042
Epoch 6, Loss: 6607.6020
Epoch 7, Loss: 4725.7599
Epoch 8, Loss: 3385.4724
Epoch 9, Loss: 2429.2009
Epoch 10, Loss: 1746.0195
Epoch 11, Loss: 1256.9572
Epoch 12, Loss: 906.2954
Epoch 13, Loss: 654.4701
Epoch 14, Loss: 473.3479
Epoch 15, Loss: 342.8702
Epoch 16, Loss: 248.7302
Epoch 17, Loss: 180.6747
Epoch 18, Loss: 131.4334
Epoch 19, Loss: 95.7505
Epoch 20, Loss: 69.8447
Epoch 21, Loss: 51.0148
Epoch 22, Loss: 37.3107
Epoch 23, Loss: 27.3199
Epoch 24, Loss: 20.0294
Epoch 25, Loss: 14.7016
Epoch 26, Loss: 10.8026
Epoch 27, Loss: 7.9472
Epoch 28, Loss: 5.8531
Epoch 29, Loss: 4.3154
Epoch 30, Loss: 3.1850
Epoch 31, Loss: 2.3531
Epoch 32, Loss: 1.7402
Epoch 33, Loss: 1.2883
Epoch 34, Loss: 0.9546
Epoch 35, Loss: 0.7081
Epoch 36, Loss: 0.5257
Epoch 37, Loss: 0.3907
Epoch 38, Loss: 0.2906
Epoch 39, Loss: 0.2164
Epoch 40, Loss: 0.1613
Epoch 41, Loss: 0.1203
Epoch 4

In [3]:

X_mean = X.mean(axis=0).values
anova = (X - X_mean) * weights  # shape (n_samples, n_features)

print("Top 10 features by ANOVA")
print(
  anova
  .abs()
  .mean(axis=0)
  .sort_values(ascending=False).head(10)
)

Top 10 features by ANOVA
15970    0.436115
14857    0.417547
21647    0.403001
34785    0.395743
39648    0.392069
47028    0.387484
10164    0.384219
2283     0.379538
5623     0.379386
14309    0.376675
dtype: float64


## Using PyTorch

Pytorch makes it really easy to parallize onto GPUs or using the parallel processing capabilities of Apple processors (MPS)

In [4]:
import torch
import timeit

# Select device (GPU if available, else CPU)
device = (
  torch.device("cuda")
  if torch.cuda.is_available()
  else torch.device("mps")
  if torch.backends.mps.is_available()
  else torch.device("cpu")
)

print(f"Using device: {device}")

X_tensor = torch.tensor(X.values.astype(np.float32), device=device)
y_tensor = torch.tensor(y.values.astype(np.float32), device=device)

# Initialize model parameters
weights = torch.zeros(n_features, device=device, requires_grad=True)

learning_rate = 1e-4
n_epochs = 50
batch_size = 32
l2_lambda = 1e-3

# Timing start
start_time = timeit.default_timer()

# Training loop
for epoch in range(n_epochs):
  perm = torch.randperm(n_samples, device=device)
  X_shuffled = X_tensor[perm]
  y_shuffled = y_tensor[perm]

  for i in range(0, n_samples, batch_size):
    X_batch = X_shuffled[i:i+batch_size]
    y_batch = y_shuffled[i:i+batch_size]

    y_pred = X_batch @ weights
    error = y_pred - y_batch

    loss = (error ** 2).mean() + l2_lambda * (weights ** 2).sum()

    # Backpropagation - this calculates the gradients for all the tensors with requires_grad=True
    loss.backward()

    # SGD step - this is where we update the weights
    with torch.no_grad():
      weights -= learning_rate * weights.grad
      weights.grad.zero_()

  # Compute full loss
  with torch.no_grad():
    y_full_pred = X_tensor @ weights
    full_loss = ((y_full_pred - y_tensor) ** 2).mean().item()
    print(f"Epoch {epoch+1}, Loss: {full_loss:.4f}")

# Timing end
end_time = timeit.default_timer()

print(f"Training completed in {end_time - start_time:.2f} seconds")


Using device: mps
Epoch 1, Loss: 23945.5039
Epoch 2, Loss: 11302.5322
Epoch 3, Loss: 5380.1025
Epoch 4, Loss: 2583.7488
Epoch 5, Loss: 1251.6373
Epoch 6, Loss: 611.0913
Epoch 7, Loss: 300.6081
Epoch 8, Loss: 148.9720
Epoch 9, Loss: 74.3519
Epoch 10, Loss: 37.3445
Epoch 11, Loss: 18.8742
Epoch 12, Loss: 9.5935
Epoch 13, Loss: 4.9027
Epoch 14, Loss: 2.5185
Epoch 15, Loss: 1.3005
Epoch 16, Loss: 0.6748
Epoch 17, Loss: 0.3517
Epoch 18, Loss: 0.1841
Epoch 19, Loss: 0.0969
Epoch 20, Loss: 0.0512
Epoch 21, Loss: 0.0272
Epoch 22, Loss: 0.0145
Epoch 23, Loss: 0.0078
Epoch 24, Loss: 0.0042
Epoch 25, Loss: 0.0023
Epoch 26, Loss: 0.0013
Epoch 27, Loss: 0.0007
Epoch 28, Loss: 0.0004
Epoch 29, Loss: 0.0002
Epoch 30, Loss: 0.0001
Epoch 31, Loss: 0.0001
Epoch 32, Loss: 0.0001
Epoch 33, Loss: 0.0000
Epoch 34, Loss: 0.0000
Epoch 35, Loss: 0.0000
Epoch 36, Loss: 0.0000
Epoch 37, Loss: 0.0000
Epoch 38, Loss: 0.0000
Epoch 39, Loss: 0.0000
Epoch 40, Loss: 0.0000
Epoch 41, Loss: 0.0000
Epoch 42, Loss: 0.0000

ALthough the total time is longer, it converges much faster (presumably because it uses better gradients), so if we were stopping on an epsilon, it would probably be faster.

In [5]:
X_tensor_mean = X_tensor.mean(axis=0)
anova_torch = (X_tensor - X_tensor_mean) * weights  # shape (n_samples, n_features)

print("Top 10 features by ANOVA")
print(
  pd.Series(
    anova_torch
    .abs()
    .mean(axis=0)
    .cpu()
    .detach()
    .numpy()
  )
  .sort_values(ascending=False)
  .head(10)
)

Top 10 features by ANOVA
15970    0.436277
14857    0.417662
21647    0.403175
34785    0.395839
39648    0.392215
47028    0.387597
10164    0.384388
2283     0.379671
5623     0.379494
14309    0.376724
dtype: float32


## Using pseudo inverse via SVD with L2 regularization



In [6]:
import numpy as np


# Timing start
start_time = timeit.default_timer()

# Compute SVD
U, S, Vt = np.linalg.svd(X, full_matrices=False)

# Compute pseudo-inverse of the singular values
l2_lambda = 1e-3  # Same as other methods
S_inv = np.diag(S / (S**2 + l2_lambda))  # Regularized version

# Compute pseudo-inverse of X
X_pinv = Vt.T @ S_inv @ U.T

# Solve for weights
w_svd = X_pinv @ y

# Timing end
end_time = timeit.default_timer()

# Check training loss
y_pred = X @ w_svd
loss = np.mean((y_pred - y) ** 2)
print(f"Training Loss (SVD pseudo-inverse): {loss:.4f}")

print(f"Training completed in {end_time - start_time:.2f} seconds")


Training Loss (SVD pseudo-inverse): 0.0000
Training completed in 3.73 seconds


In [7]:
anova_svd = (X - X_mean) * w_svd  # shape (n_samples, n_features)

print("Top 10 features by ANOVA")
print(
  anova_svd
  .abs()
  .mean(axis=0)
  .sort_values(ascending=False).head(10)
)

Top 10 features by ANOVA
15970    0.436282
14857    0.417668
21647    0.403180
34785    0.395845
39648    0.392221
47028    0.387603
10164    0.384394
2283     0.379676
5623     0.379500
14309    0.376729
dtype: float64
